We seek to reweight a portfolio of weights to be "beta-neutral" against another ETF.

In practice, we will be making our Silver Fund portfolio beta-neutral against AI ETFs

In [1]:
import numpy as np

In [2]:
import requests
import datetime as dt
import polars as pl

In [3]:
from collections import defaultdict

In [6]:
def _get_ai_etf_weights():
    ai_etf = pl.read_csv(
        'janus_henderson_ai_etf.csv',
        skip_rows=3,
        has_header=False,
    )

    column_names = [
        "underlying_security", "ticker", "cusip", "underlying_security_type",
        "strike_price", "quantity", "notional_value", "market_value", 
        "weight_percent", "current_market_value"
    ]

    ai_etf = ai_etf.select(pl.all().slice(0, 46))
    ai_etf = ai_etf.rename(dict(zip(ai_etf.columns, column_names)))

    ai_etf = ai_etf.with_columns([
        pl.col("quantity").str.replace_all(",", "").cast(pl.Int64, strict=False),
        pl.col("notional_value").str.replace_all("[$|,]", "").cast(pl.Float64, strict=False),
        pl.col("market_value").str.replace_all("[$|,]", "").cast(pl.Float64, strict=False),
        (pl.col("weight_percent").cast(pl.Float64, strict=False) / 100).alias("weight_decimal"),
        pl.col("current_market_value").str.replace_all("[$|,]", "").cast(pl.Float64, strict=False)
    ]).drop("weight_percent")

    weights_dict = defaultdict(lambda: 0.0)

    for row in ai_etf.rows(named=True):
        ticker = row.get("ticker")
        ticker = ticker.removesuffix(" US") if ticker.endswith(" US") else ticker
        weight = row.get("weight", 0.0)
        if ticker:
            weights_dict[ticker] = weight

    return weights_dict

ai_etf = _get_ai_etf_weights()

In [7]:
ai_etf

defaultdict(<function __main__._get_ai_etf_weights.<locals>.<lambda>()>,
            {'NVDA': 0.0,
             'AVGO': 0.0,
             'MSFT': 0.0,
             'GOOG': 0.0,
             'AMZN': 0.0,
             'CDNS': 0.0,
             'LRCX': 0.0,
             'MELI': 0.0,
             'PGR': 0.0,
             'INTU': 0.0,
             'WDAY': 0.0,
             'KLAC': 0.0,
             'ORCL': 0.0,
             'APP': 0.0,
             'APH': 0.0,
             'MU': 0.0,
             'DE': 0.0,
             'SNOW': 0.0,
             'ISRG': 0.0,
             'VST': 0.0,
             'NBIS': 0.0,
             'VRT': 0.0,
             'CEG': 0.0,
             'ETN': 0.0,
             'SHOP': 0.0,
             'BABA': 0.0,
             'SPOT': 0.0,
             'NVMI': 0.0,
             'LLY': 0.0,
             'HWM': 0.0,
             'IOT': 0.0,
             'SFTBY': 0.0,
             'TEAM': 0.0,
             'DDOG': 0.0,
             'BX': 0.0,
             'LITE': 0.0,
      

In [ ]:
class Portfolio:
    def __init__(self):
        """
        Initial Silver Fund weights to be re-weighted according to our views on other ETFs.
        """
        self.holdings = self._get_holdings_and_weights()
    
    @property
    def weights_array(self):
        """
        Helper to get weights as a numpy array for matrix operations.
        Ensures consistent ordering (sorted by ticker) for matrix alignment.
        """
        sorted_weights = [self.holdings[ticker] for ticker in sorted(self.holdings.keys())]
        return np.array(sorted_weights)

    def rebalance(self, V, w_f):
        """
        Rebalance our weights against another ETF.
        """
        # Convert internal dict storage to numpy array for the math
        current_weights_vec = self.weights_array
        
        vec = self._get_rebalance_vector(V, w_f)
        
        # Calculate new weights
        new_weights_vec = current_weights_vec * vec # element-wise multiplication
        
        # Update the internal dictionary with the new weights
        sorted_tickers = sorted(self.holdings.keys())
        for i, ticker in enumerate(sorted_tickers):
            self.holdings[ticker] = new_weights_vec[i]
            
        return new_weights_vec
    
    @staticmethod
    def _get_rebalance_vector(V, w_f):
        """
        Calculate the rebalance vector.
        """
        V = np.array(V)
        w_f = np.array(w_f)
        
        numerator = V @ w_f
        denominator = w_f.T @ V @ w_f
        
        return numerator / denominator

    def _get_holdings_and_weights(self):
        date_ = str(dt.date(2025, 11, 18))

        url = "https://prod-api.silverfund.byu.edu"
        endpoint = "/all-holdings/summary"
        json = {"start": date_, "end": date_, "fund": "quant_paper"}

        response = requests.post(url + endpoint, json=json)

        if not response.ok:
            raise Exception(response.text)

        holdings = pl.DataFrame(response.json()["holdings"])

        # Calculate Market Values and Weights
        holdings = holdings.with_columns([
            (pl.col("shares") * pl.col("price")).alias("market_value")
        ])
        
        total_value = holdings["market_value"].sum()
        
        holdings = holdings.with_columns([
            (pl.col("market_value") / total_value).alias("weight")
        ])

        # Convert to DefaultDict(float)
        return defaultdict(float, zip(holdings["ticker"], holdings["weight"]))

In [ ]:
class Portfolio:
    def __init__(self):
        """
        Initial Silver Fund weights to be re-weighted according to our views on other ETFs.
        """
        self.holdings = self._get_holdings_and_weights()
    
    @property
    def weights_array(self):
        """
        Helper to get weights as a numpy array for matrix operations.
        Ensures consistent ordering (sorted by ticker) for matrix alignment.
        """
        sorted_weights = [self.holdings[ticker] for ticker in sorted(self.holdings.keys())]
        return np.array(sorted_weights)

    def rebalance(self, target_etf_weights, covariance_matrix):
        """
        Rebalance our weights against another ETF.
        """
        all_tickers = sorted(set(self.holdings.keys()) | set(target_etf_weights.keys()))
        
        w_f = np.array([target_etf_weights.get(t, 0.0) for t in all_tickers])
        
        if hasattr(covariance_matrix, 'reindex'):
            V_aligned = covariance_matrix.reindex(index=all_tickers, columns=all_tickers, fill_value=0.0).values
        else:
            V_aligned = np.array(covariance_matrix)
            if V_aligned.shape[0] != len(all_tickers):
                raise ValueError(f"Covariance matrix shape {V_aligned.shape} does not match combined ticker count {len(all_tickers)}")

        new_weights_vector = self._get_rebalance_vector(V_aligned, w_f)
        
        self.holdings = defaultdict(float, dict(zip(all_tickers, new_weights_vector)))
        return self.holdings
    
    @staticmethod
    def _get_rebalance_vector(V, w_f):
        """
        Helper to calculate the rebalance vector.
        """
        V = np.array(V)
        w_f = np.array(w_f)
        
        numerator = V @ w_f
        denominator = w_f.T @ V @ w_f
        
        return numerator / denominator

    @staticmethod
    def _get_holdings_and_weights():
        """
        Helper to fetch dictionary of current holdings.
        """
        date_ = str(dt.date(2025, 11, 18))

        url = "https://prod-api.silverfund.byu.edu"
        endpoint = "/all-holdings/summary"
        json = {"start": date_, "end": date_, "fund": "quant_paper"}

        response = requests.post(url + endpoint, json=json)

        if not response.ok:
            raise Exception(response.text)

        holdings = pl.DataFrame(response.json()["holdings"])

        # Calculate Market Values and Weights
        holdings = holdings.with_columns([
            (pl.col("shares") * pl.col("price")).alias("market_value")
        ])
        
        total_value = holdings["market_value"].sum()
        
        holdings = holdings.with_columns([
            (pl.col("market_value") / total_value).alias("weight")
        ])

        # Convert to DefaultDict(float)
        return defaultdict(float, zip(holdings["ticker"], holdings["weight"]))

In [11]:
portfolio = Portfolio()
print(portfolio.holdings)

defaultdict(<class 'float'>, {'MSFT': 0.06488426463583538, 'NVDA': 0.0571939175812638, 'AAPL': 0.05408774631276599, 'LRN': 0.038955067835718754, 'AMZN': 0.03534611245872022, 'BRK B': 0.03457799070125, 'META': 0.031414711622505476, 'GOOG': 0.028001520173669633, 'GOOGL': 0.02728505596124252, 'MA': 0.022304336246674817, 'FMC': 0.021296654195160403, 'VRSN': 0.019110446558956502, 'TR': 0.018258959738749168, 'NFLX': 0.01825050440069743, 'CD': 0.017888181739048034, 'FFIV': 0.01738183267720656, 'TXN': 0.016178032487421883, 'OGN': 0.015892424879393238, 'BX': 0.015756099692507748, 'ADI': 0.015251076062036547, 'FISV': 0.01506889780541914, 'CSGP': 0.014351542354656846, 'DRS': 0.013053213770680255, 'SFM': 0.011867089807876704, 'BLK': 0.011644828678441067, 'ARCT': 0.011489867469444248, 'CVLT': 0.01120126621469999, 'ITGR': 0.01106992282158552, 'EFX': 0.010983301311085235, 'TDY': 0.010369283802674155, 'PGR': 0.01027803470851042, 'ABT': 0.010249240854604507, 'VITL': 0.010066217064181926, 'WMB': 0.00987

In [ ]:
# Test code
weights = np.array([0.3, 0.4, 0.3])
portfolio = Portfolio(weights)

V = np.array([[0.04, 0.02, 0.01],
              [0.02, 0.03, 0.015],
              [0.01, 0.015, 0.05]])

w_f = np.array([0.4, 0.3, 0.3])

new_weights = portfolio.rebalance(V, w_f)
print(f"New weights: {new_weights}")

New weights: [0.31914894 0.36595745 0.3       ]
